## Installment payment time series feature extraction
Train GRU network on installment payment time series data. Save prediction to be used as features in final training.

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
import gc

import os
print(os.listdir("../input"))
    
gc.enable()

['download_command.txt', 'application_test.csv', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


Helper functions.

In [2]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype in ["int64"]]

    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)

    return df


def scale_data(df_):
    df = df_.copy(deep=True)
    for f_ in df_.columns:
        if (df[f_].max()- df[f_].min() <=10):
            df[f_] = df[f_] - df[f_].min()
            continue
        df[f_] = df[f_] - df[f_].median()
        scale = (df[f_].quantile(0.99)-df[f_].quantile(0.01))
        if scale==0:
            scale = df[f_].max() - df[f_].min()
        df[f_] = df[f_]/scale
        if df[f_].max()>10:
            rescale = df[f_]>df[f_].quantile(0.99)
            quantile99 = df[f_].quantile(0.99)
            quantile100 = df[f_].max()
            df[f_].loc[rescale] = quantile99 + (df[f_].loc[rescale] - quantile99) * (10-quantile99)/(quantile100-quantile99)
        if df[f_].min()<-10:
            rescale = df[f_]<df[f_].quantile(0.01)
            quantile1 = df[f_].quantile(0.01)
            quantile0 = df[f_].min()
            df[f_].loc[rescale] = quantile1 + (df[f_].loc[rescale] - quantile1) * (-10-quantile1)/(quantile0-quantile1)
        df[f_] = df[f_] - df[f_].min()
    return df

Read installment data and create features.

In [3]:
inst = pd.read_csv('../input/installments_payments.csv')
inst['DAYS_ENTRY_PAYMENT_weighted'] = inst['DAYS_ENTRY_PAYMENT'] * inst['AMT_PAYMENT']
inst = inst.groupby(['SK_ID_PREV','SK_ID_CURR','NUM_INSTALMENT_NUMBER']).agg({
                                                                       'DAYS_INSTALMENT':'mean',
                                                                       'DAYS_ENTRY_PAYMENT_weighted':'sum',
                                                                       'AMT_INSTALMENT':'mean',
                                                                       'AMT_PAYMENT':'sum'})
inst['DAYS_ENTRY_PAYMENT'] = inst['DAYS_ENTRY_PAYMENT_weighted']/inst['AMT_PAYMENT']
inst = inst.reset_index()
del inst['DAYS_ENTRY_PAYMENT_weighted']
inst['AMT_PAYMENT_PERC'] = inst['AMT_PAYMENT'] / (1+inst['AMT_INSTALMENT'])
inst['DPD'] = (inst['DAYS_ENTRY_PAYMENT'] - inst['DAYS_INSTALMENT']).clip(lower=0)
inst['DBD'] = (inst['DAYS_INSTALMENT'] - inst['DAYS_ENTRY_PAYMENT']).clip(lower=0)
inst['MONTHS_BALANCE'] = (inst['DAYS_INSTALMENT']/30.4375).astype('int')
del inst['DAYS_ENTRY_PAYMENT'], inst['DAYS_INSTALMENT']
gc.collect()
#apply logarithm to make distribution more normal
inst['AMT_INSTALMENT_LOG'] = inst['AMT_INSTALMENT'].apply(np.log1p)
inst['AMT_PAYMENT_LOG'] = inst['AMT_PAYMENT'].apply(np.log1p)
inst[['AMT_INSTALMENT','AMT_PAYMENT']] = scale_data(inst[['AMT_INSTALMENT','AMT_PAYMENT']])
inst.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_NUMBER,AMT_INSTALMENT,AMT_PAYMENT,AMT_PAYMENT_PERC,DPD,DBD,MONTHS_BALANCE,AMT_INSTALMENT_LOG,AMT_PAYMENT_LOG
0,1000001,158271,1,0.044024,0.035582,0.999844,0.0,26.0,-8,8.764883,8.764883
1,1000001,158271,2,0.426463,0.344683,0.999984,0.0,6.0,-7,11.035536,11.035536
2,1000002,101962,1,0.043059,0.034802,0.999840,0.0,11.0,-52,8.742734,8.742734
3,1000002,101962,2,0.043059,0.034802,0.999840,0.0,5.0,-51,8.742734,8.742734
4,1000002,101962,3,0.043059,0.034802,0.999840,0.0,19.0,-50,8.742734,8.742734


Read target from main table.

In [4]:
data_app = pd.read_csv('../input/application_train.csv',usecols=['SK_ID_CURR','TARGET'])
data_test = pd.read_csv('../input/application_test.csv',usecols=['SK_ID_CURR'])
data_app.shape, data_test.shape

((307511, 2), (48744, 1))

In [5]:
trn_id = data_app['SK_ID_CURR'].loc[data_app.SK_ID_CURR.isin(inst.SK_ID_CURR)]
test_id = data_test['SK_ID_CURR'].loc[data_test['SK_ID_CURR'].isin(inst.SK_ID_CURR)]
trn_id.shape, test_id.shape

((291643,), (47944,))

Split train and test set. Group by ID and month to create time series.

In [6]:
inst_trn = inst.loc[inst.SK_ID_CURR.isin(trn_id)]
inst_test = inst.loc[inst.SK_ID_CURR.isin(test_id)]
num_aggregations = {
    'SK_ID_PREV': ['count'],
    'NUM_INSTALMENT_NUMBER': ['sum', 'max'],
    'AMT_INSTALMENT': ['sum','mean'],
    'AMT_PAYMENT': ['sum','mean'],
    'AMT_PAYMENT_PERC': ['mean','max'],
    'DPD': ['sum','max','mean'],
    'DBD': ['sum','max','mean'],
    'AMT_INSTALMENT_LOG': ['mean'],
    'AMT_PAYMENT_LOG': ['mean']
}
inst_trn = inst_trn.groupby(['SK_ID_CURR','MONTHS_BALANCE']).agg(num_aggregations)
inst_test = inst_test.groupby(['SK_ID_CURR','MONTHS_BALANCE']).agg(num_aggregations)
inst_trn.columns = pd.Index([e[0] + "_" + e[1].upper() for e in inst_trn.columns.tolist()])
inst_test.columns = pd.Index([e[0] + "_" + e[1].upper() for e in inst_test.columns.tolist()])

inst_trn = downcast_dtypes(inst_trn)
inst_test = downcast_dtypes(inst_test)
del inst
gc.collect()
inst_test.head()

SK_ID_PREV_COUNT  NUM_INSTALMENT_NUMBER_SUM  \
SK_ID_CURR MONTHS_BALANCE                                                
100001     -95                            1                          2   
           -94                            1                          3   
           -93                            1                          4   
           -56                            1                          1   
           -55                            1                          2   

                           NUM_INSTALMENT_NUMBER_MAX  AMT_INSTALMENT_SUM  \
SK_ID_CURR MONTHS_BALANCE                                                  
100001     -95                                     2            0.027373   
           -94                                     3            0.027373   
           -93                                     4            0.027365   
           -56                                     1            0.027160   
           -55                                     2            0.027160   

                           AMT_INSTALMENT_MEAN  AMT_PAYMENT_SUM  \
SK_ID_CURR MONTHS_BALANCE                                         
100001     -95                        0.027373         0.022124   
           -94                        0.027373         0.022124   
           -93                        0.027365         0.022118   
           -56                        0.027160         0.021951   
           -55                        0.027160         0.021951   

                           AMT_PAYMENT_MEAN  AMT_PAYMENT_PERC_MEAN  \
SK_ID_CURR MONTHS_BALANCE                                            
100001     -95                     0.022124               0.999749   
           -94                     0.022124               0.999749   
           -93                     0.022118               0.999749   
           -56                     0.021951               0.999747   
           -55                     0.021951               0.999747   

                           AMT_PAYMENT_PERC_MAX  DPD_SUM  DPD_MAX  DPD_MEAN  \
SK_ID_CURR MONTHS_BALANCE                                                     
100001     -95                         0.999749      0.0      0.0       0.0   
           -94                         0.999749     11.0     11.0      11.0   
           -93                         0.999749      0.0      0.0       0.0   
           -56                         0.999747      0.0      0.0       0.0   
           -55                         0.999747      0.0      0.0       0.0   

                           DBD_SUM  DBD_MAX  DBD_MEAN  \
SK_ID_CURR MONTHS_BALANCE                               
100001     -95                 0.0      0.0       0.0   
           -94                 0.0      0.0       0.0   
           -93                 0.0      0.0       0.0   
           -56                 6.0      6.0       6.0   
           -55                36.0     36.0      36.0   

                           AMT_INSTALMENT_LOG_MEAN  AMT_PAYMENT_LOG_MEAN  
SK_ID_CURR MONTHS_BALANCE                                                 
100001     -95                            8.289804              8.289804  
           -94                            8.289804              8.289804  
           -93                            8.289520              8.289520  
           -56                            8.281977              8.281977  
           -55                            8.281977              8.281977

Convert dataframe to 3D array (n_sample * n_time_step * n_features) for GRU network training.

In [19]:
inst_trn.fillna(-9.0, inplace=True)
inst_test.fillna(-9.0, inplace=True)
# 1. 使用 unstack 将时间序列从索引“解堆”到列，并直接填充缺失值
#    level='MONTHS_BALANCE' 指定了要转换的索引层级
#    fill_value=-9 在转换过程中一步到位地填充了缺失的时间点
train_wide = inst_trn.unstack(level='MONTHS_BALANCE', fill_value=-9)
test_wide = inst_test.unstack(level='MONTHS_BALANCE', fill_value=-9)

# 2. 获取维度信息，为重塑 (reshape) 做准备
n_train_samples = len(train_wide.index)
n_test_samples = len(test_wide.index)
n_features = len(inst_trn.columns)
n_timesteps = len(train_wide.columns) // n_features

# 3. 将2D宽数据重塑为3D数组，并交换维度以匹配GRU/LSTM的输入要求
#    目标维度: (样本数, 时间步长, 特征数)
train_x = train_wide.values.reshape(n_train_samples, n_features, n_timesteps)
train_x = np.swapaxes(train_x, 1, 2)

test_x = test_wide.values.reshape(n_test_samples, n_features, n_timesteps)
test_x = np.swapaxes(test_x, 1, 2)

# 4. train_y 的逻辑保持不变
#    假设 data_app 和 trn_id 已经定义
train_y = data_app['TARGET'].loc[data_app.SK_ID_CURR.isin(trn_id)]

# 5. 打印形状以验证
print("train_x shape:", train_x.shape)
print("test_x shape:", test_x.shape)
print("train_y shape:", train_y.shape)


train_x shape: (291643, 97, 17)
test_x shape: (47944, 97, 17)
train_y shape: (291643,)


Define GRU model. Use callback to evaluate auc metric.

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.regularizers import l2
from keras.optimizers import RMSprop, Adam

def build_model(time_step, n_features):
    model = Sequential()
    model.add(GRU(8, input_shape=(time_step, n_features))) #unit: #of neurons in each LSTM cell? input_shape=(time_step, n_features)
    model.add(Dense(1,activation='sigmoid'))
    return model

from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
import logging

class IntervalEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == (self.interval-1):
            y_pred = self.model.predict(self.X_val, verbose=0)[:,0]
            score = roc_auc_score(self.y_val, y_pred)
            print('roc score',score)
            

Training...

In [21]:
from tensorflow.keras.optimizers.legacy import Adam # for mac only

# Run a 5 fold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
oof_preds = np.zeros(train_x.shape[0])
sub_preds = np.zeros(test_x.shape[0])

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x, train_y)):
    trn_x, val_x = train_x[trn_idx], train_x[val_idx]
    trn_y, val_y = train_y.values[trn_idx], train_y.values[val_idx]

    ival = IntervalEvaluation(validation_data=(val_x, val_y), interval=5)
    model = build_model(trn_x.shape[1],trn_x.shape[2])
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4))
    model.fit(trn_x, trn_y,
              validation_data = (val_x, val_y),
              epochs=40, batch_size=2048, 
              #class_weight = {0:1,1:10},
              callbacks=[ival], verbose=1)
    
    oof_preds[val_idx] = model.predict(val_x)[:,0]
    sub_preds += model.predict(test_x)[:,0] / folds.n_splits
    
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))
  
    del model, trn_x, trn_y, val_x, val_y
    gc.collect()

Epoch 1/40


2025-10-15 14:49:26.591569: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 14:49:26.964960: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 14:49:27.295635: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


114/114 [==============================] - ETA: 0s - loss: 0.5893

2025-10-15 14:49:34.087161: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 14:49:34.155804: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


114/114 [==============================] - 9s 63ms/step - loss: 0.5893 - val_loss: 0.5421
Epoch 2/40
114/114 [==============================] - 4s 32ms/step - loss: 0.5010 - val_loss: 0.4519
Epoch 3/40
114/114 [==============================] - 4s 31ms/step - loss: 0.4119 - val_loss: 0.3751
Epoch 4/40
114/114 [==============================] - 3s 30ms/step - loss: 0.3559 - val_loss: 0.3388
Epoch 5/40
113/114 [============================>.] - ETA: 0s - loss: 0.3293

...

114/114 [==============================] - ETA: 0s - loss: 0.5057

2025-10-15 15:05:33.854866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:05:33.911361: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


114/114 [==============================] - 6s 42ms/step - loss: 0.5057 - val_loss: 0.4347
Epoch 2/40
114/114 [==============================] - 4s 34ms/step - loss: 0.3956 - val_loss: 0.3635
Epoch 3/40
114/114 [==============================] - 4s 32ms/step - loss: 0.3372 - val_loss: 0.3163
Epoch 4/40
114/114 [==============================] - 4s 31ms/step - loss: 0.3045 - val_loss: 0.2970
Epoch 5/40
113/114 [============================>.] - ETA: 0s - loss: 0.2935

2025-10-15 15:05:49.891020: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-15 15:05:49.932264: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


roc score 0.5282985207295254
114/114 [==============================] - 11s 99ms/step - loss: 0.2935 - val_loss: 0.2902
Epoch 6/40
114/114 [==============================] - 4s 32ms/step - loss: 0.2888 - val_loss: 0.2877
Epoch 7/40
114/114 [==============================] - 4s 31ms/step - loss: 0.2869 - val_loss: 0.2863
Epoch 8/40
114/114 [==============================] - 4s 31ms/step - loss: 0.2857 - val_loss: 0.2853
Epoch 9/40
114/114 [==============================] - 4s 32ms/step - loss: 0.2850 - val_loss: 0.2848
Epoch 10/40
114/114 [==============================] - 12s 109ms/step - loss: 0.2847 - val_loss: 0.2845
Epoch 11/40
114/114 [==============================] - 4s 34ms/step - loss: 0.2844 - val_loss: 0.2843
Epoch 12/40
114/114 [==============================] - 4s 32ms/step - loss: 0.2842 - val_loss: 0.2841
Epoch 13/40
114/114 [==============================] - 4s 31ms/step - loss: 0.2841 - val_loss: 0.2839
Epoch 14/40
114/114 [==============================] - 4s 32ms/ste

Save model prediction to disk.

In [22]:
inst_score_train = pd.DataFrame({'inst_score':oof_preds}, index=trn_id)
inst_score_test = pd.DataFrame({'inst_score':sub_preds}, index=test_id)             
inst_score_train.to_csv('../output/inst_score_train.csv')
inst_score_test.to_csv('../output/inst_score_test.csv')